## Import TensorFlow and other libraries

It's good practice to use a validation split when developing your model. Let's use 80% of the images for training, and 20% for validation.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing import image

#----------------------------------------------------------------------------------------------------------------
batch_size = 7  # Load 7 images at a time
img_height = 224
img_width = 224
#----------------------------------------------------------------------------------------------------------------
# Paths to your image files
img_paths = [
    "D:/CNN-With/data/CDM/CDMls206 - Copy - Copy.jpg",
    "D:/CNN-With/data/HT/ht206 - Copy - Copy.jpg",
    "D:/CNN-With/data/MDMV/MDMVls1 - Copy (2).jpg",
    "D:/CNN-With/data/NCLB/NCLBls1 - Copy (2).jpg",
    "D:/CNN-With/data/SCLB/SCLBls1 - Copy.jpg",
    "D:/CNN-With/data/SCMV/SCMVls1 - Copy.jpg",
    "D:/CNN-With/data/SR/SRls1 - Copy - Copy.jpg"
]

# Load and preprocess images
images = []
for img_path in img_paths:
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)  # Convert to numpy array
    images.append(img_array)

# Convert the list of images to a numpy array
images = np.array(images)

# Add batch dimension (batch_size, height, width, channels)
# Images are already in the right shape (7, 224, 224, 3), no need to expand dimensions

# Normalize the images (if the model requires it, adjust as needed)
# images = images / 255.0  # Uncomment if model was trained with normalized images

#----------------------------------------------------------------------------------------------------------------
class_names = ['CDM', 'HT', 'MDMV', 'NCLB', 'SCLB', 'SCMV', 'SR']  # Class names for the images
print("Class Names:", class_names)
#----------------------------------------------------------------------------------------------------------------

# Define the path where the model is saved (in H5 format)
saved_model_path = "D:/CNN-With/Trained_Model_Cnn/model.h5"

# Load the model
model = tf.keras.models.load_model(saved_model_path)

# Print the model summary
model.summary()
#----------------------------------------------------------------------------------------------------------------

# Predict using the loaded model
predictions = model.predict(images)

# If the model has logits as output, apply softmax
if 'from_logits' in model.loss.get_config() and model.loss.get_config()['from_logits']:
    predictions = tf.nn.softmax(predictions).numpy()

# Create a DataFrame to show the prediction probabilities for all images
pred_df = pd.DataFrame(predictions, columns=class_names)

# Display the prediction results for all images
print("Prediction results for the images:")
print(pred_df)

# Assuming true labels for the images (replace with actual labels)
true_labels = ["CDM", "HT", "MDMV", "NCLB", "SCLB", "SCMV", "SR"]  # Replace with actual labels
predicted_labels = pred_df.idxmax(axis=1).tolist()  # Get the predicted label for each image
confidences = pred_df.max(axis=1).tolist()  # Get the confidence of each prediction

# Print the true labels, predicted labels, and their respective confidence
for i in range(batch_size):
    print(f"\nImage {i+1}:")
    print(f"True Label: {true_labels[i]}")
    print(f"Predicted Label: {predicted_labels[i]}")
    print(f"Prediction Confidence: {confidences[i]:.4f}")

#----------------------------------------------------------------------------------------------------------------
# Create confusion matrix for the batch
cm = confusion_matrix(true_labels, predicted_labels, labels=class_names)

# Convert confusion matrix to DataFrame for better readability
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

# Print the confusion matrix
print("Confusion Matrix for the batch:\n")
print(cm_df)

# Calculate Precision, Recall, and F1-Score for the batch
precision = precision_score(true_labels, predicted_labels, average=None, labels=class_names)
recall = recall_score(true_labels, predicted_labels, average=None, labels=class_names)
f1 = f1_score(true_labels, predicted_labels, average=None, labels=class_names)

# Display metrics
metrics_df = pd.DataFrame({
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1
}, index=class_names)

print("\nPrecision, Recall, and F1-Score for each class:")
print(metrics_df)

# Optionally, calculate average precision, recall, and F1-Score for the batch
average_precision = precision_score(true_labels, predicted_labels, average='macro')
average_recall = recall_score(true_labels, predicted_labels, average='macro')
average_f1 = f1_score(true_labels, predicted_labels, average='macro')

print(f"\nAverage Precision: {average_precision:.4f}")
print(f"Average Recall: {average_recall:.4f}")
print(f"Average F1-Score: {average_f1:.4f}")

#----------------------------------------------------------------------------------------------------------------
# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix for the Batch of Images')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Plotting the metrics table as a heatmap (similar to confusion matrix)
plt.figure(figsize=(8, 4))
sns.heatmap(metrics_df, annot=True, fmt='.4f', cmap='Blues', cbar=False, annot_kws={"size": 12},
            xticklabels=metrics_df.columns, yticklabels=metrics_df.index, linewidths=1, linecolor='black')
plt.title("Precision, Recall, and F1-Score for each Class")
plt.xlabel('Metrics')
plt.ylabel('Classes')
plt.show()